In [1]:
!pip install amplpy pyomo -q
!python -m amplpy.modules install coin highs scip gcg -q

In [2]:
from amplpy import modules
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from pyomo.environ import *


import matplotlib.pyplot as plt
import networkx as nx
import requests

In [6]:
from pyomo.environ import RangeSet
from folium import Map, Marker, Icon
from folium.plugins import AntPath
import folium
import csv
import os

class VehicleRoutingProblem:
    def __init__(self, clients_file, depots_file, vehicles_file, case_number, output_dir,
                 num_clients, num_vehicles, num_depots):
        self.clients_file = clients_file
        self.depots_file = depots_file
        self.vehicles_file = vehicles_file
        self.case_number = case_number
        self.output_dir = output_dir

        # Parámetros dependientes del caso
        self.num_clients = num_clients
        self.num_vehicles = num_vehicles
        self.num_depots = num_depots

        # Atributos de rangos y ubicaciones
        self.clients_id = None
        self.vehicles_r = None
        self.depot_origin = None
        self.locations = None

        # Otros atributos
        self.model = None
        self.data = {}
        self.locations_data = None  # Para almacenar ubicaciones combinadas
        self.vehicles_info = None  # Para almacenar la información combinada de vehículos
        self.costos_distancia = None
        self.costos_duracion = None

        # Asegurar que el directorio de salida exista
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def initialize_ranges(self):
        """Inicializa los conjuntos de rangos basados en los parámetros proporcionados."""
        self.clients_id = RangeSet(1, self.num_clients)
        self.vehicles_r = RangeSet(1, self.num_vehicles)  # Asumí que los vehículos empiezan en 1
        self.depot_origin = RangeSet(1, self.num_depots)
        self.locations = RangeSet(1, self.num_clients + self.num_depots)
        
        print("Conjuntos inicializados:")
        print(f"Clientes (ID): {list(self.clients_id)}")
        print(f"Vehículos (R): {list(self.vehicles_r)}")
        print(f"Depósitos (Origen): {list(self.depot_origin)}")
        print(f"Ubicaciones: {list(self.locations)}")

    def load_data(self):
        """Carga datos de los archivos proporcionados."""
        self.data['clients'] = pd.read_csv(self.clients_file)
        self.data['depots'] = pd.read_csv(self.depots_file)
        self.data['vehicles'] = pd.read_csv(self.vehicles_file)

    def preview_data(self, data_key, num_samples=5):
        """Muestra detalles del archivo cargado, según el tipo de datos."""
        if data_key in self.data:
            print(f"{data_key.capitalize()} Data Shape: {self.data[data_key].shape}")
            print(self.data[data_key].sample(num_samples))
        else:
            print(f"El conjunto de datos '{data_key}' no está cargado. Asegúrate de llamar a `load_data` primero.")

    def combine_locations(self):
        """Combina ubicaciones de depósitos y clientes en un solo DataFrame."""
        if 'clients' not in self.data or 'depots' not in self.data:
            raise ValueError("Los datos de clientes y depósitos deben cargarse antes de combinar ubicaciones.")
        
        self.locations_data = pd.concat([
            self.data['depots'][["LocationID", "Longitude", "Latitude"]],
            self.data['clients'][["LocationID", "Longitude", "Latitude"]]
        ], axis=0).reset_index(drop=True)
        
        print(f"Ubicaciones combinadas: {self.locations_data.shape}")
        print(self.locations_data.sample(15))

    def merge_vehicle_data(self):
        """Combina datos de vehículos con información adicional."""
        vehicle_additional_data = pd.DataFrame({
            'Vehicle': ['Gas Car', 'EV', 'Drone'],
            'Freight Rate [COP/km]': [5000, 4000, 500],
            'Time Rate [COP/min]': [500, 500, 500],
            'Daily Maintenance [COP/day]': [30000, 21000, 3000],
            'Recharge/Fuel Cost [COP/gal or kWh]': [16000, 0, 220.73],
            'Recharge/Fuel Time [min/10 percent charge]': [0.1, 0, 2.0],
            'Avg. Speed [km/h]': [0, 0, 40.0],
            'Gas Efficiency [km/gal]': [10.0, 0, 0],
            'Electricity Efficiency [kWh/km]': [0, 0.15, 0.15]
        })

        if 'vehicles' not in self.data:
            raise ValueError("Los datos de vehículos no están cargados. Llama a `load_data` primero.")
        
        self.vehicles_info = self.data['vehicles'].merge(
            vehicle_additional_data, 
            left_on='VehicleType', 
            right_on='Vehicle', 
            how='left'
        )

        print(f"Datos combinados de vehículos: {self.vehicles_info.shape}")
        print(self.vehicles_info.sample(5))

    def fill_missing_values(self, fill_value=0):
        """Reemplaza valores nulos en `vehicles_info` por el valor especificado."""
        if self.vehicles_info is None:
            raise ValueError("Los datos combinados de vehículos no están disponibles. Llama a `merge_vehicle_data` primero.")
        
        self.vehicles_info.fillna(fill_value, inplace=True)
        print("Valores nulos reemplazados en `vehicles_info`.")
        print(self.vehicles_info.sample(6))

    def fetch_osrm_matrices(self):
        """Obtiene las matrices de distancia y duración usando la API OSRM."""
        if self.locations_data is None:
            raise ValueError("Las ubicaciones deben combinarse primero. Llama a `combine_locations`.")

        # Crear el parámetro de ubicación para la URL de OSRM
        result = ";".join(f"{row['Longitude']},{row['Latitude']}" for _, row in self.locations_data.iterrows())
        url = f"http://router.project-osrm.org/table/v1/driving/{result}?annotations=distance,duration"
        
        try:
            response = requests.get(url, timeout=60)  # Aumenté el timeout a 60 segundos
            response.raise_for_status()
            print("Respuesta de la API OSRM recibida correctamente.")
            print(response.json().keys())
            self.costos_distancia = np.array(response.json()["distances"]) / 1000  # Convertir a kilómetros
            self.costos_duracion = np.array(response.json()["durations"]) / 60  # Convertir a minutos
            print(f"Dimensiones: Distancia {self.costos_distancia.shape}, Duración {self.costos_duracion.shape}")
        except requests.exceptions.RequestException as e:
            print("Error al obtener las matrices de OSRM:", e)
            raise

    def build_model(self):
        """Construye el modelo de optimización."""
        model = pyo.ConcreteModel()
        model.C = pyo.Set(initialize=self.data['clients']['LocationID'].tolist())
        model.D = pyo.Set(initialize=self.data['depots']['LocationID'].tolist())
        model.V = pyo.Set(initialize=self.vehicles_info.index.tolist())
        model.N = pyo.Set(initialize=list(model.C) + list(model.D))

        # Parámetros ajustados
        model.capacity = pyo.Param(model.V, initialize=dict(zip(self.vehicles_info.index, self.vehicles_info['Capacity'])))
        model.range = pyo.Param(model.V, initialize=dict(zip(self.vehicles_info.index, self.vehicles_info['Range'])))
        model.cost_per_km = pyo.Param(model.V, initialize=dict(zip(self.vehicles_info.index, self.vehicles_info['Freight Rate [COP/km]'])))
        model.cost_per_min = pyo.Param(model.V, initialize=dict(zip(self.vehicles_info.index, self.vehicles_info['Time Rate [COP/min]'])))
        model.cost_per_combustible = pyo.Param(model.V, initialize=dict(zip(self.vehicles_info.index, self.vehicles_info['Recharge/Fuel Cost [COP/gal or kWh]'])))
        model.cost_per_load = pyo.Param(initialize=100)

        model.demand = pyo.Param(model.C, initialize=dict(zip(self.data['clients']["LocationID"], self.data['clients']["Product"])))

        # Costos por distancia y duración
        dist_dict = {
            (self.locations_data.iloc[i]["LocationID"], self.locations_data.iloc[j]["LocationID"]): self.costos_distancia[i, j]
            for i in range(len(self.locations_data))
            for j in range(len(self.locations_data))
            if i != j
        }
        model.distances = pyo.Param(model.N, model.N, initialize=dist_dict, default=0)

        dura_dict = {
            (self.locations_data.iloc[i]["LocationID"], self.locations_data.iloc[j]["LocationID"]): self.costos_duracion[i, j]
            for i in range(len(self.locations_data))
            for j in range(len(self.locations_data))
            if i != j
        }
        model.durations = pyo.Param(model.N, model.N, initialize=dura_dict, default=0)

        model.maintenance_cost = pyo.Param(model.V, initialize=dict(zip(self.vehicles_info.index, self.vehicles_info['Daily Maintenance [COP/day]'])))

        # Variables de decisión
        model.x = pyo.Var(model.N, model.N, model.V, domain=pyo.Binary)
        model.u = pyo.Var(model.C, model.V, within=pyo.NonNegativeIntegers)
        model.v_use = pyo.Var(model.V, within=pyo.Binary)
        model.carga = pyo.Var(model.N, model.N, model.V, domain=pyo.NonNegativeReals)
        
        # Función objetivo
        def objective_rule(model):
            #Costo por distancia
            cost_distance = sum(model.cost_per_km[v] * model.distances[i, j] * model.x[i, j, v]
                                for v in model.V for i in model.N for j in model.N if i != j)
            #Costo por duración
            cost_duration = sum(model.cost_per_min[v] * model.durations[i, j] * model.x[i, j, v]
                                for v in model.V for i in model.N for j in model.N if i != j)
            #Costo de mantenimiento
            cost_maintenance = sum(model.maintenance_cost[v] * model.v_use[v] for v in model.V)
            
            #Costo de carga por demanda cubierta
            cost_load = sum(model.cost_per_load* model.demand[i] * model.x[i, j, v]
                        for v in model.V for i in model.C for j in model.N if i != j)
            #Costo de combustible
            cost_fuel = sum(
                model.cost_per_combustible[v] * model.distances[i, j] * model.x[i, j, v] * (1 / self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"])
                for v in model.V for i in model.N for j in model.N if i != j and self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"] != 0
            )
            #Costo por tiempo de recarga de combustible
            cost_recharge = sum(
                model.cost_per_min[v] * (model.distances[i, j] * model.x[i, j, v] * (1 / self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"])) * (1 / self.vehicles_info.loc[v, "Recharge/Fuel Time [min/10 percent charge]"])
                for v in model.V for i in model.N for j in model.N if i != j and self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"] != 0
            )
            
            return cost_distance + cost_duration + cost_maintenance + cost_load + cost_fuel + cost_recharge

        model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

        # Restricciones
        def visit_once_rule(model, i):
            return sum(model.x[i, j, v] for v in model.V for j in model.N if i != j) == 1
        model.visit_once = pyo.Constraint(model.C, rule=visit_once_rule)

        def flow_conservation_rule(model, v, i):
            return sum(model.x[i, j, v] for j in model.N if i != j) == sum(model.x[j, i, v] for j in model.N if i != j)
        model.flow_conservation = pyo.Constraint(model.V, model.N, rule=flow_conservation_rule)

        def vehicle_used_rule(model, v):
            return model.v_use[v] * len(model.N) >= sum(model.x[i, j, v] for i in model.N for j in model.N if i != j)
        model.vehicle_used = pyo.Constraint(model.V, rule=vehicle_used_rule)

        def vehicle_range_rule(model, v):
            return sum(model.distances[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
        model.vehicle_range = pyo.Constraint(model.V, rule=vehicle_range_rule)

        def depot_salida_rule(model, v):
            return sum(model.x[d, j, v] for d in model.D for j in model.C) == 1
        model.depot_salida = pyo.Constraint(model.V, rule=depot_salida_rule)

        def return_to_depot_rule(model, v, d):
            return sum(model.x[d, j, v] for j in model.N if j != d) == sum(model.x[j, d, v] for j in model.N if j != d)
        model.return_to_depot = pyo.Constraint(model.V, model.D, rule=return_to_depot_rule)

        def vehicle_capacity_rule_01(model, v):
            return sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.capacity[v]
        model.vehicle_capacity_01 = pyo.Constraint(model.V, rule=vehicle_capacity_rule_01)

        def vehicle_capacity_rule(model, i, j, v):
            return model.carga[i, j, v] <= model.capacity[v] * model.x[i, j, v]
        model.vehicle_capacity = pyo.Constraint(model.N, model.N, model.V, rule=vehicle_capacity_rule)

        def satisfy_demand_rule(model, i):
            return sum(model.carga[i, j, v] for j in model.N for v in model.V if j != i) == model.demand[i]
        model.satisfy_demand = pyo.Constraint(model.C, rule=satisfy_demand_rule)

        def subtour_elimination_rule(model, v, i, j):
            if i != j and i not in model.D and j not in model.D:
                return model.u[i, v] - model.u[j, v] + len(model.C) * model.x[i, j, v] <= len(model.C) - 1
            else:
                return pyo.Constraint.Skip
        model.subtour_elimination = pyo.Constraint(model.V, model.C, model.C, rule=subtour_elimination_rule)

        self.model = model

    def visualize_routes(self, map_filename=None):
        """
        Genera un mapa interactivo mostrando las rutas seleccionadas para cada vehículo.

        Parámetros:
            map_filename (str): Nombre del archivo HTML donde se guardará el mapa.
        """
        if self.model is None:
            raise ValueError("El modelo no está construido. Llama a `build_model` primero.")
        
        # Asegurarse de que el modelo haya sido resuelto
        if not hasattr(self.model, 'x') or any(pyo.value(var) is None for var in self.model.x.values()):
            raise ValueError("El modelo no ha sido resuelto o no se han asignado valores a las variables de decisión.")
        
        selected_routes = []
        for v in self.model.V:
            routes_for_v = [
                (i, j) for i in self.model.N for j in self.model.N 
                if i != j and np.round(pyo.value(self.model.x[i, j, v])) == 1
            ]
            print(f"Rutas seleccionadas para el vehículo {v}: {routes_for_v}")
            selected_routes.append(routes_for_v)
        
        # Crear un diccionario de coordenadas
        coordenadas = {}
        for idx, row in self.locations_data.iterrows():
            location_id = row["LocationID"]
            coordenadas[location_id] = {"coor": (row["Latitude"], row["Longitude"])}
        
        # Asignar colores a los vehículos
        colors = ['green', 'orange', 'darkblue', 'darkred', 'red', 'purple']
        node_colors = {}
        for v_idx, v in enumerate(self.model.V):
            for route in selected_routes[v]:
                i, j = route
                idx_color = v_idx % len(colors)
                color = colors[idx_color]
                
                if i not in node_colors and i > self.num_depots:
                    coordenadas[i]["v"] = v
                    node_colors[i] = color
                if j not in node_colors and j > self.num_depots:
                    coordenadas[j]["v"] = v
                    node_colors[j] = color
        
        # Crear el mapa centrado en la primera ubicación
        first_location = next(iter(coordenadas))
        mapa = folium.Map(location=coordenadas[first_location]["coor"], zoom_start=11)
        
        for idx in coordenadas:
            lat, lon = coordenadas[idx]["coor"]
            if idx <= self.num_depots:
                # Depósitos
                folium.Marker(
                    location=(lat, lon),
                    popup=f"Depósito: {idx}",
                    icon=folium.Icon(color='gray', icon='warehouse', prefix="fa"),
                ).add_to(mapa)
            else:
                # Clientes
                color = node_colors.get(idx, "black")
                vehicle = coordenadas[idx].get("v", -1)
                folium.Marker(
                    location=(lat, lon),
                    popup=(
                        f"Cliente: {idx - self.num_depots}, "
                        f"LocationID: {idx}, "
                        f"Demanda: {self.model.demand[idx]} uds, "
                        f"Vehículo: {vehicle}"
                    ),
                    icon=folium.Icon(color=color, icon='home', prefix="fa"),
                ).add_to(mapa)
        
        # Dibujar las rutas usando AntPath
        for v_idx, v in enumerate(self.model.V):
            for route in selected_routes[v]:
                i, j = route
                if i in coordenadas and j in coordenadas:
                    folium.plugins.AntPath(
                        locations=[coordenadas[i]["coor"], coordenadas[j]["coor"]],
                        tooltip=f"Vehículo {v}: Distancia: {self.costos_distancia[i - 1, j - 1]:.2f} km",
                        color=colors[v_idx % len(colors)],
                    ).add_to(mapa)
        
        # Guardar y mostrar el mapa
         # Asignar un nombre de archivo basado en el caso si no se proporciona
        if map_filename is None:
            map_filename = f"mapa_caso{self.case_number}.html"
    
        # Guardar y mostrar el mapa
        mapa_path = os.path.join(self.output_dir, map_filename)
        mapa.save(mapa_path)
        print(f"Mapa guardado en {mapa_path}")
        
        # Opcional: Retornar el objeto mapa si se desea mostrar en un entorno interactivo
        mapa
        return mapa

    def generate_reports(self, tipo_caso="estandar", numero_caso=1):
        """
        Genera archivos de reporte incluyendo rutas secuenciales, valor de la función objetivo
        y costos operacionales.

        Parámetros:
            tipo_caso (str): Tipo de caso (por ejemplo, "estandar").
            numero_caso (int): Número del caso.
        """
        if self.model is None:
            raise ValueError("El modelo no está construido. Llama a `build_model` primero.")
        
        # Asegurarse de que el modelo haya sido resuelto
        if not hasattr(self.model, 'x') or any(pyo.value(var) is None for var in self.model.x.values()):
            raise ValueError("El modelo no ha sido resuelto o no se han asignado valores a las variables de decisión.")
        
        # Nombre del archivo de rutas
        archivo_rutas = os.path.join(
            self.output_dir,
            f"G12-caso-{tipo_caso}-{numero_caso}-ruta.csv"
        )
        
        # Generar las rutas secuenciales
        rutas = []
        for v in self.model.V:
            # Encuentra el depósito de inicio para el vehículo
            ruta_actual = []
            nodo_actual = None
            for d in self.model.D:
                for j in self.model.N:
                    if pyo.value(self.model.x[d, j, v]) > 0.5:  # El vehículo sale de este depósito
                        ruta_actual = [d]  # Comienza la ruta desde el depósito
                        nodo_actual = j
                        break
                if nodo_actual is not None:
                    break
            
            if nodo_actual is None:
                print(f"Vehículo {v} no tiene una ruta asignada.")
                continue  # Salta al siguiente vehículo
            
            # Construye la ruta secuencialmente
            while nodo_actual not in self.model.D:  # Hasta que regrese al depósito
                ruta_actual.append(nodo_actual)
                siguiente_nodo = None
                for j in self.model.N:
                    if pyo.value(self.model.x[nodo_actual, j, v]) > 0.5:
                        siguiente_nodo = j
                        break
                if siguiente_nodo is None:
                    break  # No hay más nodos, termina la ruta
                nodo_actual = siguiente_nodo
        
            # Agrega la ruta al depósito final
            if nodo_actual in self.model.D:
                ruta_actual.append(nodo_actual)
            else:
                print(f"Vehículo {v} no regresó a un depósito.")
        
            # Genera las transiciones para el archivo
            for i in range(len(ruta_actual) - 1):
                rutas.append([v, ruta_actual[i], ruta_actual[i + 1]])
        
        # Escribir el archivo CSV
        try:
            with open(archivo_rutas, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["ID-Vehiculo", "ID-Origen", "ID-Destino"])  # Cabecera
                writer.writerows(rutas)
            print(f"Archivo de rutas generado: {archivo_rutas}")
        except Exception as e:
            print(f"Error al escribir el archivo de rutas: {e}")
            raise
        
        # Nombre del archivo de la función objetivo
        nombre_archivo_objetivo = os.path.join(
            self.output_dir,
            f"Caso{numero_caso}_Objetivo.txt"
        )
        
        # Obtener el valor de la función objetivo
        valor_objetivo = pyo.value(self.model.objective)
        
        # Guardar el valor en un archivo
        try:
            with open(nombre_archivo_objetivo, mode='w') as file:
                file.write(f"Valor de la función objetivo: {valor_objetivo}\n")
            print(f"Archivo de función objetivo generado: {nombre_archivo_objetivo}")
        except Exception as e:
            print(f"Error al escribir el archivo de función objetivo: {e}")
            raise
        
        # Nombre del archivo de costos operacionales
        nombre_archivo_costos = os.path.join(
            self.output_dir,
            f"Caso{numero_caso}_Costos.txt"
        )
        
        # Calcular costos individuales
        costo_distancia = sum(
            self.model.cost_per_km[v] * self.model.distances[i, j] * pyo.value(self.model.x[i, j, v])
            for v in self.model.V for i in self.model.N for j in self.model.N if i != j
        )
        
        costo_duracion = sum(
            self.model.cost_per_min[v] * self.model.durations[i, j] * pyo.value(self.model.x[i, j, v])
            for v in self.model.V for i in self.model.N for j in self.model.N if i != j
        )
        
        costo_mantenimiento = sum(
            self.model.maintenance_cost[v] * pyo.value(self.model.v_use[v]) for v in self.model.V
        )
        
        costo_carga = sum(self.model.cost_per_load * self.model.demand[i] * pyo.value(self.model.x[i, j, v]) for v in self.model.V for i in self.model.C for j in self.model.N if i != j)
        
        costo_fuel = 0
        costo_recharge = 0
        for v in self.model.V:
            for i in self.model.N:
                for j in self.model.N:
                    if i != j:
                        if self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"] != 0:
                            costo_fuel += (
                                self.model.cost_per_combustible[v] * 
                                self.model.distances[i, j] * 
                                pyo.value(self.model.x[i, j, v]) * 
                                (1 / self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"])
                            )
                            costo_recharge += (
                                self.model.cost_per_min[v] * 
                                (
                                    (self.model.distances[i, j] * pyo.value(self.model.x[i, j, v]) * 
                                     (1 / self.vehicles_info.loc[v, "Gas Efficiency [km/gal]"])
                                    ) * 
                                    (1 / self.vehicles_info.loc[v, "Recharge/Fuel Time [min/10 percent charge]"])
                                )
                            )
                        else:
                            costo_fuel += 0
                            costo_recharge += 0
        
        # Escribir reporte en el archivo
        try:
            with open(nombre_archivo_costos, mode='w') as file:
                file.write("Reporte de Costos Operacionales:\n")
                file.write(f"Costo por distancia recorrida (COP): {costo_distancia}\n")
                file.write(f"Costo por duración del trayecto (COP): {costo_duracion}\n")
                file.write(f"Costo de mantenimiento diario (COP): {costo_mantenimiento}\n")
                file.write(f"Costo por carga transportada (COP): {costo_carga}\n")
                file.write(f"Costo por carga de combustible (COP): {costo_fuel}\n")
                file.write(f"Costo por tiempo de carga de combustible (COP): {costo_recharge}\n")
                file.write(f"Costo total (función objetivo) (COP): {costo_distancia + costo_duracion + costo_mantenimiento + costo_carga + costo_fuel + costo_recharge}\n")
            print(f"Archivo de costos operacionales generado: {nombre_archivo_costos}")
        except Exception as e:
            print(f"Error al escribir el archivo de costos operacionales: {e}")
            raise

class modelSolver:
    def __init__(self, model):
        self.model = model

    def solve_model(self):
        """Resuelve el modelo utilizando el solver HiGHS."""
        solver_name = "highs"  # Nombre del solver
        solver = pyo.SolverFactory(solver_name, solver_io="nl")  # Usar el solver HiGHS

        # Configurar opciones avanzadas del solver
        solver.options['parallel'] = 'on'  # Habilitar paralelismo
        solver.options['time_limit'] = 3600  # Límite de tiempo (1 hora)
        solver.options['presolve'] = 'on'  # Habilitar preprocesamiento
        solver.options['mip_rel_gap'] = 0.00  # Gap relativo del 1% (para MIP)

        # Resolver el modelo
        result = solver.solve(self.model, tee=True)

        # Imprimir resultados
        print(result.solver.status)
        print(result.solver.termination_condition)

        # Verificar el estado de la solución
        if result.solver.termination_condition == pyo.TerminationCondition.optimal:
            print("Optimal solution found.")
        elif result.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
            print("Time limit reached, solution may be suboptimal.")
        else:
            print(f"Solver terminated with condition: {result.solver.termination_condition}")

        # Mostrar el resultado completo
        print(result)
            




# CASO 1

In [5]:
# Crear instancia del problema para el caso 1
vrp_case_1 = VehicleRoutingProblem(
    clients_file=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_1_base\Clients.csv",
    depots_file=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_1_base\Depots.csv",
    vehicles_file=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_1_base\Vehicles.csv",
    case_number=1,
    output_dir=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\resultados",
    num_clients=24,
    num_vehicles=11, #Ingresa el número de vehículos -1
    num_depots=12
)

# Inicializar rangos y cargar datos
vrp_case_1.initialize_ranges()
vrp_case_1.load_data()

# Previsualizar datos de clientes
vrp_case_1.preview_data('clients')

# Combinar ubicaciones de clientes y depósitos
vrp_case_1.combine_locations()

# Combinar datos de vehículos con información adicional
vrp_case_1.merge_vehicle_data()

# Reemplazar valores nulos en los datos combinados de vehículos
vrp_case_1.fill_missing_values()

# Obtener matrices de distancia y duración desde la API OSRM
vrp_case_1.fetch_osrm_matrices()

vrp_case_1.build_model()
            
# Resolver el modelo
solver = modelSolver(vrp_case_1.model)
solver.solve_model()

caso=1
# Generar la visualización de rutas
vrp_case_1.visualize_routes(map_filename=f"mapa_Caso{caso}.html")
            
# Generar los reportes
vrp_case_1.generate_reports(tipo_caso="estandar", numero_caso=1)


Conjuntos inicializados:
Clientes (ID): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Vehículos (R): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Depósitos (Origen): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Ubicaciones: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Clients Data Shape: (24, 5)
    ClientID  LocationID  Product  Longitude  Latitude
22        23          35       15 -74.116526  4.731407
21        22          34       18 -74.086836  4.625426
1          2          14       15 -74.075571  4.687821
13        14          26       15 -74.121920  4.725912
5          6          18       17 -74.120838  4.662137
Ubicaciones combinadas: (36, 3)
    LocationID  Longitude  Latitude
21          22 -74.090411  4.557380
8            9 -74.095472  4.735973
5            6 -74.124002  4.650463
2            3 -74.038548  4.792926
4            5 -74.138263  4.607707
25    

NameError: name 'costo_carga' is not defined

# CASO 2

In [ ]:
# Crear instancia del problema para el caso 1
vrp_case_2 = VehicleRoutingProblem(
    clients_file=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_2_cost\Clients.csv",
    depots_file=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_2_cost\Depots.csv",
    vehicles_file=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_2_cost\Vehicles.csv",
    case_number=2,
    output_dir=r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\resultados",
    num_clients=30,
    num_vehicles=5, #Ingresa el número de vehículos -1
    num_depots=12
)

# Inicializar rangos y cargar datos
vrp_case_2.initialize_ranges()
vrp_case_2.load_data()

# Previsualizar datos de clientes
vrp_case_2.preview_data('clients')

# Combinar ubicaciones de clientes y depósitos
vrp_case_2.combine_locations()

# Combinar datos de vehículos con información adicional
vrp_case_2.merge_vehicle_data()

# Reemplazar valores nulos en los datos combinados de vehículos
vrp_case_2.fill_missing_values()

# Obtener matrices de distancia y duración desde la API OSRM
vrp_case_2.fetch_osrm_matrices()

vrp_case_2.build_model()
            
# Resolver el modelo
solver = modelSolver(vrp_case_2.model)
solver.solve_model()

caso=2
# Generar la visualización de rutas
vrp_case_2.visualize_routes(map_filename=f"mapa_Caso{caso}.html")
            
# Generar los reportes
vrp_case_2.generate_reports(tipo_caso="estandar", numero_caso=2)

Conjuntos inicializados:
Clientes (ID): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Vehículos (R): [1, 2, 3, 4, 5]
Depósitos (Origen): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Ubicaciones: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
Clients Data Shape: (30, 5)
    ClientID  LocationID  Product  Longitude  Latitude
28        29          41        8 -74.183218  4.607013
22        23          35       13 -74.120522  4.601890
5          6          18        7 -74.143682  4.566156
4          5          17        5 -74.097386  4.592061
25        26          38        8 -74.086313  4.631637
Ubicaciones combinadas: (42, 3)
    LocationID  Longitude  Latitude
16          17 -74.097386  4.592061
37          38 -74.086313  4.631637
40          41 -74.183218  4.607013
35          36 -74.088648  4.626697
15          16